In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [10]:
# read data and clean it

In [3]:
df = pd.read_csv("../data/MY2015-2019 Fuel Consumption Ratings.csv")

In [4]:
df

,Model year,Make,Model,Vehicle class,Engine size (L),Cylinders,Transmission,Fuel type,City (L/100 km),Highway (L/100 km),Combined (L/100 km),Combined (mpg),CO2 emissions (g/km),CO2 rating,Smog rating
0,2015,Acura,ILX,Compact,2.0,4,AS5,Z,9.7,6.7,8.3,34,191,NaN,NaN
1,2015,Acura,ILX,Compact,2.4,4,M6,Z,10.8,7.4,9.3,30,214,NaN,NaN
2,2015,Acura,ILX Hybrid,Compact,1.5,4,AV7,Z,6.0,6.1,6.1,46,140,NaN,NaN
3,2015,Acura,MDX SH-AWD,Sport utility vehicle: Small,3.5,6,AS6,Z,12.7,9.1,11.1,25,255,NaN,NaN
4,2015,Acura,RDX AWD,Sport utility vehicle: Small,3.5,6,AS6,Z,12.1,8.7,10.6,27,244,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5426,2019,Volvo,XC40 T5 AWD,Sport utility vehicle: Small,2.0,4,AS8,Z,10.3,7.5,9.0,31,211,5.0,5.0
5427,2019,Volvo,XC60 T5 AWD,Sport utility vehicle: Small,2.0,4,AS8,Z,11.3,8.5,10.0,28,233,5.0,5.0
5428,2019,Volvo,XC60 T6 AWD,Sport utility vehicle: Small,2.0,4,AS8,Z,11.8,8.6,10.4,27,242,5.0,7.0
5429,2019,Volvo,XC90 T5 AWD,Sport utility vehicle: Standard,2.0,4,AS8,Z,11.3,8.5,10.0,28,233,5.0,5.0


In [5]:
df.Transmission.unique()
df['Fuel type'].unique()

array(['Z', 'D', 'X', 'E', 'N'], dtype=object)

In [12]:
#making a master copy 
cdf = df.copy(deep=True) 
# keeping specific column
cdf = cdf[['Engine size (L)',
          'Cylinders',
          'City (L/100 km)',
          'Highway (L/100 km)',
          'Combined (L/100 km)',
          'Combined (mpg)',
          'CO2 emissions (g/km)',
         ]]
# https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-pandas-dataframe
cdf["model"] = df["Make"].astype(str) + "_" + df["Model"].astype(str) + "_" + df["Transmission"].astype(str) + "_" + df["Engine size (L)"].astype(str) + "_" + df["Fuel type"].astype(str) + "_" + df["Model year"].astype(str)

cdf

,Engine size (L),Cylinders,City (L/100 km),Highway (L/100 km),Combined (L/100 km),Combined (mpg),CO2 emissions (g/km),model
0,2.0,4,9.7,6.7,8.3,34,191,Acura_ILX_AS5_2.0_Z_2015
1,2.4,4,10.8,7.4,9.3,30,214,Acura_ILX_M6_2.4_Z_2015
2,1.5,4,6.0,6.1,6.1,46,140,Acura_ILX Hybrid_AV7_1.5_Z_2015
3,3.5,6,12.7,9.1,11.1,25,255,Acura_MDX SH-AWD_AS6_3.5_Z_2015
4,3.5,6,12.1,8.7,10.6,27,244,Acura_RDX AWD_AS6_3.5_Z_2015
...,...,...,...,...,...,...,...,...
5426,2.0,4,10.3,7.5,9.0,31,211,Volvo_XC40 T5 AWD_AS8_2.0_Z_2019
5427,2.0,4,11.3,8.5,10.0,28,233,Volvo_XC60 T5 AWD_AS8_2.0_Z_2019
5428,2.0,4,11.8,8.6,10.4,27,242,Volvo_XC60 T6 AWD_AS8_2.0_Z_2019
5429,2.0,4,11.3,8.5,10.0,28,233,Volvo_XC90 T5 AWD_AS8_2.0_Z_2019


In [13]:
# check if any duplicated
cdf.duplicated(subset='model').sum()

0

In [14]:
# print the duplicates
ids = cdf["model"]
cdf[ids.isin(ids[ids.duplicated()])].sort_values("model")

,Engine size (L),Cylinders,City (L/100 km),Highway (L/100 km),Combined (L/100 km),Combined (mpg),CO2 emissions (g/km),model


In [15]:
# Merge duplicate rows by taking the mean
# Identify duplicate rows based on the string column
duplicate_rows = cdf[cdf.duplicated(subset=['model'])]
duplicate_rows

,Engine size (L),Cylinders,City (L/100 km),Highway (L/100 km),Combined (L/100 km),Combined (mpg),CO2 emissions (g/km),model


In [16]:
# Aggregate duplicate rows by taking the mean of numerical columns -> merged_rows
merged_rows = duplicate_rows.groupby('model').agg('mean').reset_index()
merged_rows

,model,Engine size (L),Cylinders,City (L/100 km),Highway (L/100 km),Combined (L/100 km),Combined (mpg),CO2 emissions (g/km)


In [17]:
# Drop duplicate rows
cdf.drop_duplicates(subset=['model'], keep='first', inplace=True)

In [18]:
cdf.duplicated(subset='model').sum()

0

In [ ]:
# TODO Replace duplicate rows with mean values 
# for index, row in merged_rows.iterrows():
#     string_value = row['StringColumn']
#     mean_value = row['NumericColumn']
#     df.loc[df['StringColumn'] == string_value, 'NumericColumn'] = mean_value

In [96]:
# # Concatenate merged rows with non-duplicate rows
# result = pd.concat([cdf.drop_duplicates(subset=['model']), merged_rows], ignore_index=True)
# resultv

In [19]:
sample_size = cdf.size
sample_size
# df = df.sample(n=sample_size, replace=False, random_state=490)

rcdf = cdf.sample(n=5000, random_state=50)
rcdf.head()

rcdf = rcdf.reset_index()
rcdf = rcdf.drop('index',axis=1)
rcdf.head()

,Engine size (L),Cylinders,City (L/100 km),Highway (L/100 km),Combined (L/100 km),Combined (mpg),CO2 emissions (g/km),model
0,2.5,4,9.1,7.1,8.2,34,192,Nissan_Rogue_AV_2.5_X_2019
1,3.5,6,13.8,9.7,12.0,24,276,Honda_Pilot 4WD_A5_3.5_X_2015
2,1.6,4,8.8,6.4,7.7,37,179,Kia_Rio_M6_1.6_X_2016
3,1.4,4,8.6,6.5,7.6,37,178,Volkswagen_Golf SportWagen_AS8_1.4_X_2019
4,5.0,8,15.2,10.2,13.0,22,306,Genesis_G90 AWD_AS8_5.0_Z_2019


In [20]:
from sklearn.preprocessing import StandardScaler
# Standardize the numeric features
scaler = StandardScaler()
cdf[['Engine size (L)', 'Cylinders', 'City (L/100 km)', 'Highway (L/100 km)', 'Combined (L/100 km)', 'Combined (mpg)']] = scaler.fit_transform(cdf[['Engine size (L)', 'Cylinders', 'City (L/100 km)', 'Highway (L/100 km)', 'Combined (L/100 km)', 'Combined (mpg)']])


cdf[['CO2 emissions (g/km)']] = scaler.fit_transform(cdf[['CO2 emissions (g/km)']]) * -1

In [21]:
# Selecting only the numeric columns for similarity calculation
features = ['Engine size (L)', 'Cylinders', 'City (L/100 km)', 'Highway (L/100 km)', 'Combined (L/100 km)', 'Combined (mpg)', 'CO2 emissions (g/km)']
numeric_df = cdf[features]

In [22]:
from sklearn.metrics.pairwise import euclidean_distances

# Calculate Euclidean distance matrix
euclidean_dist_matrix = euclidean_distances(numeric_df, numeric_df)

In [23]:
# Function to recommend similar models based on Euclidean distance
def get_recommendations(model, top_n = 3):
    idx = cdf.index[cdf['model'] == model].tolist()[0]
    distances_with_idx = list(enumerate(euclidean_dist_matrix[idx]))
    distances_with_idx = sorted(distances_with_idx, key=lambda x: x[1])
    similar_models = [(cdf.iloc[i]['model'], - distance) for i, distance in distances_with_idx[1:top_n+1]]
    return similar_models

In [24]:
# Example usage
model_to_recommend_for = 'Volvo_XC40 T5 AWD_AS8_2.0_Z_2019'
recommendations = get_recommendations(model_to_recommend_for)
print(f"Recommendations for {model_to_recommend_for}: {recommendations}")

Recommendations for Volvo_XC40 T5 AWD_AS8_2.0_Z_2019: [('MINI_John Cooper Works Clubman ALL4_AS8_2.0_Z_2018', -0.029646780732288665), ('MINI_Cooper S 3 Door_M6_2.0_Z_2018', -0.05057306396867175), ('MINI_Cooper S 5 Door_M6_2.0_Z_2018', -0.05057306396867175)]


In [25]:
# Example usage 2
model_to_recommend_for = 'Acura_ILX_AS5_2.0_Z_2015'
recommendations = get_recommendations(model_to_recommend_for)
print(f"Recommendations for {model_to_recommend_for}: {recommendations}")

Recommendations for Acura_ILX_AS5_2.0_Z_2015: [('Hyundai_Elantra_AS6_2.0_X_2015', -0.0), ('Hyundai_Elantra Coupe_M6_2.0_X_2015', -0.0), ('Kia_Forte_AS6_2.0_X_2015', -0.0)]


In [26]:
# Example usage 2
model_to_recommend_for = 'Genesis_G90 AWD_AS8_5.0_Z_2019'
recommendations = get_recommendations(model_to_recommend_for)
print(f"Recommendations for {model_to_recommend_for}: {recommendations}")

Recommendations for Genesis_G90 AWD_AS8_5.0_Z_2019: [('Genesis_G90 AWD_AS8_5.0_Z_2018', -0.0), ('Genesis_G90 AWD_AS8_5.0_Z_2019', -0.0), ('Jaguar_F-TYPE Project 7 Convertible_AS8_5.0_Z_2016', -0.11662157886645907)]


In [ ]:
#### END

In [6]:
# import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.preprocessing import StandardScaler

# # Sample DataFrame
# data = {
#     'Model': ['Toyota Camry', 'Honda Civic', 'Ford Fusion', 'Tesla Model 3'],
#     'Carbon_Emission': [100, 120, 110, 50],  # Lower values indicate better
#     'Fuel_Efficiency': [30, 35, 32, 40]       # Higher values indicate better
# }
# df = pd.DataFrame(data)

# # Feature scaling
# scaler = StandardScaler()
# df[['Carbon_Emission', 'Fuel_Efficiency']] = scaler.fit_transform(df[['Carbon_Emission', 'Fuel_Efficiency']])

# # Calculate cosine similarity matrix
# features = ['Carbon_Emission', 'Fuel_Efficiency']
# cosine_sim = cosine_similarity(df[features], df[features])

# # Function to recommend similar items based on cosine similarity
# def get_recommendations(model, top_n=3):
#     idx = df.index[df['Model'] == model].tolist()[0]
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:top_n+1]
#     similar_models = [df.iloc[s[0]]['Model'] for s in sim_scores]
#     return similar_models

# # Example usage
# model_to_recommend_for = 'Toyota Camry'
# recommendations = get_recommendations(model_to_recommend_for)
# print(f"Recommendations for {model_to_recommend_for}: {recommendations}")

In [1]:
# # trying to make a plot
# tdf = cdf.copy(deep=True)
# # using pca to compress the features into 2d
# pca = PCA(2)

# #Transform the data
# tdf = pca.fit_transform(tdf)

# df.shape

# # # keeping specific column
# # tdf = cdf[[
# #           'Combined (mpg)',
# #           'CO2 emissions (g/km)',
# #          ]]
# # tdf

In [2]:
#  #Initialize the class object
# kmeans = KMeans(n_clusters= 10)
 
# #predict the labels of clusters.
# label = kmeans.fit_predict(tdf)
 
# print(label)

In [3]:
# # plotting start
# import matplotlib.pyplot as plt
 
# #filter rows of original data
# filtered_label0 = tdf[label == 0]
 
# #plotting the results
# plt.scatter(filtered_label0[:,0] , filtered_label0[:,1])
# plt.show()

In [4]:
# #filter rows of original data
# filtered_label2 = tdf[label == 2]
 
# filtered_label8 = tdf[label == 8]
 
# #Plotting the results
# plt.scatter(filtered_label2[:,0] , filtered_label2[:,1] , color = 'red')
# plt.scatter(filtered_label8[:,0] , filtered_label8[:,1] , color = 'black')
# plt.show()

In [5]:
# #Getting unique labels
 
# u_labels = np.unique(label)
 
# #plotting the results:
 
# for i in u_labels:
#     plt.scatter(tdf[label == i , 0] , tdf[label == i , 1] , label = i)
# plt.legend()
# plt.show()